<h1>Home Assignment injunctions!</h1>
<li>No Loops. Use only elementwise operations</li>
<li>Your code should work for any data in the provided format. Data values (example, customer ids, product ids, number of customers, etc.) should NEVER appear in your code!</li>

<h1>Problem 1:</h1>
Write a function that reads timeseries pricing data from a file into a pandas dataframe and then groups the data as follows:
<li>The arguments to the function are the filename and a threshold number
<li>The function reads the data in the file and creates a new column "pct_change" with the one day percent change
<li>Then groups the data into four categories:
<ul>
<li>"High+" if the percent change is greater than the threshold 
<li>"Low+" if the percent change is zero or positive and less than or equal to the threshold% 
<li>"Low-" if the percent change is negative but greater than or equal to -1 * the threshold
<li>"High-" if the percent change is less than -1 * the threshold
</ul>
<li>The function should return a dataframe that contains three columns (count, mean, stdev) and four index values (High+, High-, Low+, Low-)
    <p><b>Note: </b>we have to deal with nan percent changes. Make sure that you don't count a NaN in any of the four categories! (see https://pandas.pydata.org/docs/reference/api/pandas.isna.html)  <p>
For the sample data your function should return a dataframe with the following values for a threshold of 1.0:

<pre>
        count	mean	stdev
High+	870	2.162834	1.301745
High-	666	-2.291770	1.370657
Low+	870	0.463122	0.284869
Low-	883	-0.445557	0.276355

</pre>

    

<h3>read_csv</h3>
The pandas <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html">read_csv</a> function reads data from a delimited file into a pandas dataframe.



<h2>Sample data</h2>
Use the attached AAPL.csv file

In [1]:
import pandas as pd
import numpy as np

In [2]:
def change_groups(datafile,threshold):
    #Your code goes here
    
    #read the data in the file
    df = pd.read_csv(datafile)
    
    #calculate the one day percent change
    p_change = df.iloc[:,1].pct_change()
    p_change = p_change.to_frame()
    p_change.columns = ["pct"]
    p_change["pct"] = p_change["pct"]*100
    
    #group the data into four categories
    p_change["cat"] = p_change.apply(lambda x: 
                                     "High+" if x.pct > threshold
                                     else "High-" if x.pct < -1*threshold
                                     else "Low+" if x.pct >= 0 and x.pct <= threshold
                                     else "Low-" if x.pct < 0 and x.pct >= -1*threshold
                                     else "NaN",
                                     axis = 1)
    
    #analysis by four categories
    output = p_change.groupby(by = "cat").count()
    output.columns = ["count"]
    output["mean"] = p_change.groupby(by = "cat").mean()
    output["stdev"] = p_change.groupby(by = "cat").std()
    output.index.name = None
    
    print(output[0:4])

In [3]:
df = pd.read_csv("AAPL.csv")
p_change = df.iloc[:,1].pct_change()
p_change = p_change.to_frame()
p_change.columns = ["pct"]
p_change["pct"] = p_change["pct"]*100

threshold=1.0
p_change["cat"] = p_change.apply(lambda x: 
                                 "High+" if x.pct > threshold
                                 else "High-" if x.pct < -1*threshold
                                 else "Low+" if x.pct >= 0 and x.pct <= threshold
                                 else "Low-" if x.pct < 0 and x.pct >= -1*threshold
                                 else "NaN",
                                 axis = 1)

p_change

,pct,cat
0,NaN,NaN
1,0.172915,Low+
2,-1.590661,High-
3,-0.184872,Low-
4,0.664864,Low+
...,...,...
3285,2.350044,High+
3286,1.006306,High+
3287,-0.470075,Low-
3288,1.480337,High+


In [4]:
#Test your code
change_groups("AAPL.csv",1.0)

       count      mean     stdev
High+    870  2.162834  1.301745
High-    666 -2.291770  1.370657
Low+     870  0.463122  0.284869
Low-     883 -0.445557  0.276355


<h1>Problem 2</h1>
A manufacturer has data on orders from customers and product prices in two dataframes (see below). They want to use this data to answer the following questions:
<ol>
    <li>Which customer is responsible for the most revenue</li>
    <li>Which customer is responsible for the highest profit</li>
    <li>Which product is responsible for the highest (dollar) profit</li>
    <li>Which customer and product combination is responsible for the most orders</li>
</ol>
<p>
Obviously, your code should work for any actual data values and pandas dataframes of any length!
<p>For the data below: your answers should be:
    
<pre>
Customer with most profit: 005
Customer with most revenue: 007
Product with most profit: 011
Customer 001 with product 010 with 4 orders is the most ordered customer product pair
</pre>


<h2>Useful functions:</h2>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html">pd.sort_values</a> </li>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html">pandas dataframe join</a> the last example on the linked page is probably what you need here!</li>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html">pd.groupby</a></li>

In [5]:
import pandas as pd
import numpy as np
order_data = pd.DataFrame([["001","010",24],
                           ["007","012",35],
                           ["001","011",15],
                           ["005","010",30],
                           ["007","011",17],
                           ["005","011",81],
                           ["001","010",32],
                           ["007","012",89],
                           ["001","010",16],
                           ["001","010",33]],columns=["customer","product","amount"])
products = pd.DataFrame([['010',22.3,17.2],
                        ['011',11.7,5.5],
                        ['012',62.5,61.4]],columns=['product','price','cost'])                     

In [6]:
order_data

,customer,product,amount
0,001,010,24
1,007,012,35
2,001,011,15
3,005,010,30
4,007,011,17
5,005,011,81
6,001,010,32
7,007,012,89
8,001,010,16
9,001,010,33


In [7]:
products

,product,price,cost
0,010,22.3,17.2
1,011,11.7,5.5
2,012,62.5,61.4


In [8]:
#Your code goes here

#join order_data and products into one dataframe
##and calculate revenue and profit 
data = order_data.join(products.set_index("product"), on="product")
data = data.assign(revenue = data.amount * data.price)
data = data.assign(profit = data.amount * (data.price - data.cost))
data

,customer,product,amount,price,cost,revenue,profit
0,001,010,24,22.3,17.2,535.2,122.4
1,007,012,35,62.5,61.4,2187.5,38.5
2,001,011,15,11.7,5.5,175.5,93.0
3,005,010,30,22.3,17.2,669.0,153.0
4,007,011,17,11.7,5.5,198.9,105.4
5,005,011,81,11.7,5.5,947.7,502.2
6,001,010,32,22.3,17.2,713.6,163.2
7,007,012,89,62.5,61.4,5562.5,97.9
8,001,010,16,22.3,17.2,356.8,81.6
9,001,010,33,22.3,17.2,735.9,168.3


In [9]:
#check which customer is responsible for the most revenue and highest profit
customer_value = data.groupby(by = "customer").sum(numeric_only = True).sort_values(by = "revenue")
customer_value

,amount,price,cost,revenue,profit
customer,,,,,
005,111,34.0,22.7,1616.7,655.2
001,120,100.9,74.3,2517.0,628.5
007,141,136.7,128.3,7948.9,241.8


In [10]:
#check which product is responsible for the highest (dollar) profit
product_profit = data.groupby(by = "product").sum(numeric_only = True).sort_values(by = "profit")
product_profit

,amount,price,cost,revenue,profit
product,,,,,
012,124,125.0,122.8,7750.0,136.4
010,135,111.5,86.0,3010.5,688.5
011,113,35.1,16.5,1322.1,700.6


In [11]:
#check which customer and product combination is responsible for the most order
pair_frequency = data.groupby(["customer","product"]).count().sort_values(by = "amount")
pair_frequency

amount  price  cost  revenue  profit
customer product                                      
001      011           1      1     1        1       1
005      010           1      1     1        1       1
         011           1      1     1        1       1
007      011           1      1     1        1       1
         012           2      2     2        2       2
001      010           4      4     4        4       4

In [12]:
#print the answer:
print("Customer with most profit: " + customer_value["profit"].idxmax())
print("Customer with most revenue: " + customer_value["revenue"].idxmax())
print("Product with most profit: " + product_profit["profit"].idxmax())
print("Customer " + pair_frequency["amount"].idxmax()[0] 
      + " with product " + pair_frequency["amount"].idxmax()[1] 
      +  " with " + str(pair_frequency["amount"].max()) 
      + " orders is the most ordered customer product pair")

Customer with most profit: 005
Customer with most revenue: 007
Product with most profit: 011
Customer 001 with product 010 with 4 orders is the most ordered customer product pair
